In [31]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

### load the data summed up

In [32]:
years = [str(year) for year in range(2013, 2025)]

df = pd.read_excel("data.xlsx", sheet_name=years)

data_list = []

for year, df in df.items():
    # Füge das Jahr als neue Spalte hinzu
    df['Jahr'] = year
    
    # Füge den DataFrame der Liste hinzu
    data_list.append(df)

# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenführen
final_df = pd.concat(data_list, ignore_index=True)



### clean the data

In [33]:
df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]] # drop * in the data
df.columns = df.columns.str.replace('*', '', regex=False)# just to be sure
df = df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo
clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

# smack together the similar col names 
merged_df = df[['Jahr']].copy()
for clean_name, original_cols in clean_columns.items():
    if len(original_cols) > 1:
        merged_df[clean_name] = df[original_cols].sum(axis=1)
    else:
        merged_df[clean_name] = df[original_cols[0]]
df = merged_df.fillna(value=0) # use if needed for plotting

### transform

In [34]:
df = df[df['Variable'].isin(['Zufriedenheit'])]
df = df[df['Variable'] == 'Zufriedenheit'].drop(columns='Variable')
df = df[df["Category"] != "Gesamt (%)"]
df = df.groupby(['Jahr', 'Category']).sum().reset_index()
df

,Jahr,Category,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Ernährungswissenschaften,Master Agrarökonomie und Betriebsmanagement,Master Nutztierwissenschaften,...,Master Transition Management,Master Ökotrophologie,Master Agrobiotechnologie,Master Insect Biotechnology and Bioresources,Bachelor Umweltmanagement,Master Ernährungsökonomie,Master Nutzpflanzenwissenschaften,Master Oenologie/Weinwirtschaft,Master Nachhaltige Ernährungswirtschaft,Master Sustainable Transition
0,2013,Anzahl,79.0,88.0,0.0,137.0,73.0,70.0,11.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2013,eher zufrieden (%),71.0,83.0,0.0,77.0,63.0,56.0,55.0,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2013,gar nicht zufrieden (%),0,1.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2013,mittelmäßig zufrieden (%),13.0,3.0,0.0,4.0,15.0,16.0,9.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013,sehr zufrieden (%),4.0,3.0,0.0,3.0,7.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2024,eher zufrieden (%),73,66.0,86.0,73.0,72.0,72.0,0.0,0.0,...,68.0,75.0,64.0,50.0,0.0,0.0,67.0,0.0,92.0,67.0
68,2024,gar nicht zufrieden (%),0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,2024,mittelmäßig zufrieden (%),10,22.0,7.0,12.0,12.0,14.0,0.0,0.0,...,5.0,8.0,9.0,0.0,0.0,0.0,17.0,0.0,0.0,17.0
70,2024,sehr zufrieden (%),17,10.0,7.0,10.0,10.0,6.0,0.0,0.0,...,23.0,17.0,27.0,42.0,0.0,0.0,17.0,0.0,8.0,0.0


In [35]:
anzahl_row = df[df['Category'] == 'Anzahl']
for year in df['Jahr'].unique():
    gesamtzahlen = anzahl_row[anzahl_row['Jahr'] == year].drop(columns=['Jahr', 'Category']).values.flatten()

    # Sicherstellen, dass alle relevanten Spalten als numerische Werte vorliegen
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Umwandlung in numerisch, fehlerhafte Werte werden zu NaN

    # Gehe alle Zeilen durch, die nicht 'Anzahl' sind
    for idx, row in df[df['Jahr'] == year].iterrows():
        if row['Category'] != 'Anzahl':
            # Extrahiere die Prozentsätze der aktuellen Zeile
            relative_prozent = row.drop(['Jahr', 'Category']).values
            # Berechne die absoluten Werte
            absolute_werte = (relative_prozent / 100) * gesamtzahlen
            # Überschreibe die Prozentsätze mit den berechneten absoluten Werten
            df.loc[idx, df.columns[2:]] = absolute_werte
            df.round(0)
df = df[df['Category'] != 'Anzahl']
df['sum']= df.sum(axis=1, numeric_only=True).round(0)
df = df.get(key=['Jahr', 'Category', 'sum'])

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_4512/3221710608.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

In [56]:
df_rel = df.pivot(index="Jahr", columns="Category")
df_rel.columns = df_rel.columns.droplevel()
df_rel = df_rel.apply(pd.to_numeric, errors='coerce')


In [62]:
df_rel = (
    df_rel.div(df_rel.sum(axis=1), axis=0).mul(100)
)

In [ ]:
df_rel

Category,gar nicht zufrieden (%),weniger zufrieden (%),mittelmäßig zufrieden (%),eher zufrieden (%),sehr zufrieden (%)
Jahr,,,,,
2013,0.351494,14.938489,11.775044,69.068541,3.866432
2014,0.736648,21.178637,8.471455,65.745856,3.867403
2015,0.000000,15.178571,11.607143,70.982143,2.232143
2016,0.343053,15.094340,8.576329,69.811321,6.174957
2017,1.306241,12.191582,6.531205,71.117562,8.853411
2018,0.000000,10.965630,5.564648,72.340426,11.129296
2019,0.361882,8.685163,6.755127,72.617612,11.580217
2020,0.000000,6.363636,5.064935,74.155844,14.415584
2021,0.367197,9.791922,5.507956,70.012240,14.320685


In [75]:

# Reihenfolge der Kategorien festlegen
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

# DataFrame umwandeln für Plotly
df_melted = df_rel.reset_index().melt(id_vars="Jahr", var_name="Category", value_name="sum")
df_melted.rename(columns={"index": "Jahr"}, inplace=True)

# Sicherstellen, dass 'Category' eine kategorische Variable mit der richtigen Reihenfolge ist
df_melted['Category'] = pd.Categorical(df_melted['Category'], categories=category_order, ordered=True)

# Sortiere den DataFrame nach Jahr und Category
df_melted = df_melted.sort_values(by=['Jahr', 'Category'])

# Gestapeltes Balkendiagramm mit Plotly erstellen
fig = px.bar(df_melted, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="plotly_white", 
             barmode="stack",
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Relativer Anteil der Antworten',
                 'Category': 'Kategorien'
             })

# Diagramm anzeigen
fig.show()

In [80]:

# Reihenfolge der Kategorien festlegen
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

# DataFrame umwandeln für Plotly
df_melted = df_rel.reset_index().melt(id_vars="Jahr", var_name="Category", value_name="sum")
df_melted.rename(columns={"index": "Jahr"}, inplace=True)

# Sicherstellen, dass 'Category' eine kategorische Variable mit der richtigen Reihenfolge ist
df_melted['Category'] = pd.Categorical(df_melted['Category'], categories=category_order, ordered=True)

# Sortiere den DataFrame nach Jahr und Category
df_melted = df_melted.sort_values(by=['Jahr', 'Category'])

# Gestapeltes Balkendiagramm mit Plotly erstellen
fig = px.bar(df_melted, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="plotly_white", 
             barmode="group",
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Relativer Anteil der Antworten',
                 'Category': 'Kategorien'
             })
fig.update_layout(height=700)
# Diagramm anzeigen
fig.show()

In [38]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

# Stelle sicher, dass 'Category' eine kategorische Variable mit der richtigen Reihenfolge ist
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

# Sortiere den DataFrame nach Jahr und Category
df = df.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz", 
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Beteiligung an der Befragung',
                 'Category': 'Kategorien'
             })
fig.show()

In [81]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

# Stelle sicher, dass 'Category' eine kategorische Variable mit der richtigen Reihenfolge ist
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

# Sortiere den DataFrame nach Jahr und Category
df = df.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz",
             barmode="group", 
             
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Beteiligung an der Befragung',
                 'Category': 'Kategorien'
             })
fig.update_layout(height=700)
fig.show()